# Download Data for baseline

In [13]:
import wget
url = r"https://tickettagger.blob.core.windows.net/datasets/dataset-labels-top3-30k-real.txt"
output = r"../dataset/dataset-labels-top3-30k-real.txt"
wget.download(url,output)

url = r"https://gist.githubusercontent.com/rafaelkallis/6aa281b00d73d77fc843bd34f8184854/raw/8c10ebf2fd6f937f8667c660ea33d122bac739eb/issues.txt"
output = r"../dataset/dataset_balanced.txt"
wget.download(url,output)

'../dataset/dataset-labels-top3-30k-real (1).txt'

## look at pivot table of data from other repos

In [90]:
import pandas as pd
# custom data set
data_path = r"..\dataset\multiple_projects_benchmark_data.csv"
data_out_path = r"..\dataset\CLEAN_multiple_projects_benchmark_data.csv"
df = pd.read_csv(data_path)
df = df.dropna()
df.head()


,f0_,repo_name
0,__label__bug Single most recently added Custom...,https://api.github.com/repos/qgis/QGIS
1,__label__bug layer style effects not compatibl...,https://api.github.com/repos/qgis/QGIS
2,__label__bug QGIS 3.10.0 fails to launch on Wi...,https://api.github.com/repos/qgis/QGIS
3,__label__bug TIN interpolation in Model Builde...,https://api.github.com/repos/qgis/QGIS
4,__label__bug Map Tip Position / Follow the Hyp...,https://api.github.com/repos/qgis/QGIS


In [91]:
# import pandas as pd

# data_path = r"..\dataset\dataset_balanced.txt"
# data_out_path = r"..\dataset\CLEAN_balanced_data.csv"

# df = pd.read_csv(data_path,names=["f0_"])
# df = df.dropna()
# df = df.drop(index=0).reset_index()
# df.head()

In [131]:
import pandas as pd

data_path = r"..\dataset\dataset-labels-top3-30k-real.txt"
data_out_path = r"..\dataset\CLEAN_UNbalanced_data.csv"

df = pd.read_csv(data_path,names=["f0_"])
df = df.dropna()
df = df.drop(index=0).reset_index()
df.head()
df.drop(columns=['index'])

,f0_
0,__label__enhancement Subpasses in MSL and HLSL...
1,__label__enhancement Collect new discovered no...
2,__label__enhancement Lock property : several p...
3,__label__bug Return default location type labe...
4,__label__enhancement Randomize: remove sliders
...,...
29994,__label__enhancement [TW-241] new column forma...
29995,__label__bug Error When using the example. ...
29996,__label__bug CorrelationId should be generated...
29997,__label__enhancement screenshot sidecar should...


In [132]:
# df['labels'] = df['f0_'].str.split(" ").map(lambda x: x[0])
# df = df.rename(columns={'f0_':'descriptions'})
# print(df.columns)
# df.head()

In [133]:
# df.groupby(['repo_name','labels']).agg({'descriptions':'count'})
# df.groupby(['labels']).agg({'descriptions':'count'})

In [134]:
# Cleaning the text
df = df['f0_'].str.split(r'(__label__enhancement)|(__label__bug)|(__label__question)', expand=True)
# retrieve data to separate dataframe for each labels
enh_df = df[df[1] == '__label__enhancement'][[1, 4]]
bug_df = df[df[2] == '__label__bug'][[2, 4]]
question_df = df[df[3] == '__label__question'][[3, 4]]

# assign column names to each df
enh_df.columns = ['labels', 'descriptions']
bug_df.columns = ['labels', 'descriptions']
question_df.columns = ['labels', 'descriptions']

print(enh_df.head())
print(bug_df.head())
print(question_df.head())

# remove the prefix '__label__' string from column labels
enh_df['labels'] = enh_df['labels'].str.replace('__label__', '')
bug_df['labels'] = bug_df['labels'].str.replace('__label__', '')
question_df['labels'] = question_df['labels'].str.replace('__label__', '')

# strip away and leading & trailing spaces
enh_df['descriptions'] = enh_df['descriptions'].str.strip(' ')
bug_df['descriptions'] = bug_df['descriptions'].str.strip(' ')
question_df['descriptions'] = question_df['descriptions'].str.strip(' ')

# concatenate each df to single dataframe
df = pd.concat([enh_df, bug_df, question_df])

df.head()


                 labels                                       descriptions
0  __label__enhancement   Subpasses in MSL and HLSL Sorry for all the i...
1  __label__enhancement   Collect new discovered nodes Collect new adde...
2  __label__enhancement        Lock property : several properties at once 
4  __label__enhancement                         Randomize: remove sliders 
8  __label__enhancement   feature request: local invoke inside action c...
         labels                                       descriptions
3  __label__bug   Return default location type label if custom ...
5  __label__bug   [TW-417] JSON export has extra commas (on 2.1...
6  __label__bug   System.AccessViolationException: my program t...
7  __label__bug   Topic mining fails with UnicodeEncodeError ``...
9  __label__bug   Preflight and non-batching requests throw an ...
                labels                                       descriptions
11   __label__question   Avoiding the default NotEmpty::IS_EMPTY valid...


,labels,descriptions
0,enhancement,Subpasses in MSL and HLSL Sorry for all the is...
1,enhancement,Collect new discovered nodes Collect new added...
2,enhancement,Lock property : several properties at once
4,enhancement,Randomize: remove sliders
8,enhancement,feature request: local invoke inside action co...


In [135]:
import pandas as pd
from sklearn.model_selection import train_test_split
import neattext.functions as nfx
import re

In [136]:
# remove any non-english samples
# reference: https://stackoverflow.com/a/150078/9960491
reg_obj = re.compile(r'[^\u0000-\u007F]+', re.UNICODE)
def is_english(desc):
    return (False if reg_obj.match(desc) else True)

df['is_english'] = df['descriptions'].apply(is_english)
df = df[df['is_english']][['labels', 'descriptions']]

In [137]:
# remove the stopwords, emojis from the text and convert it into lower case
def neatify_text(text):
    text = str(text).lower()
    text = nfx.remove_stopwords(text)
    text = nfx.remove_emojis(text)
#     text = nfx.remove_puncts(text)
    return text

df['descriptions'] = df['descriptions'].apply(neatify_text)

In [138]:
# split dataset to train and dev samples
RANDOM_SEED = 12
x_train, x_test, y_train, y_test = train_test_split(df['descriptions'], df['labels'],
                                                 test_size=0.3,
                                                 random_state=RANDOM_SEED)
df_train = pd.concat([x_train, y_train], axis=1).reset_index(drop=True)
df_eval = pd.concat([x_test, y_test], axis=1).reset_index(drop=True)


In [139]:
df.groupby(['labels']).agg({'descriptions':'count'})
print(df.head())

        labels                                       descriptions
0  enhancement  subpasses msl hlsl sorry issues  given followi...
1  enhancement  collect new discovered nodes collect new added...
2  enhancement                         lock property : properties
4  enhancement                          randomize: remove sliders
8  enhancement  feature request: local invoke inside action co...


In [140]:
# # # save train and eval dataframes to csv
# df_train.to_csv(r"../dataset/Mult_train.csv")
# df_eval.to_csv(r"../dataset/Mult_eval.csv")
# df.to_csv(data_out_path)

In [141]:
# save train and eval dataframes to csv
# df_train.to_csv(r"../dataset/balanced_train.csv")
# df_eval.to_csv(r"../dataset/balanced_eval.csv")
# df.to_csv(data_out_path)

In [142]:
# save train and eval dataframes to csv
df_train.to_csv(r"../dataset/unbalanced_train.csv",index=False)
df_eval.to_csv(r"../dataset/unbalanced_eval.csv",index=False)
df.to_csv(data_out_path,index=False)